<a href="https://colab.research.google.com/github/mobraine/Natural-Language-Understanding/blob/main/%E2%80%9CNL2DS_F2021_Assignment_6_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Language Modeling

In this part, let's generate text using a trigram language model.

Go to https://drive.google.com/drive/folders/1pR0koayRSgXfTD72HZUHN14uec0SrnXy?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>


Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive.

*Copy function may not work. If so, manually copy the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


When you run the `ls` command below, you should see these folders.




In [ ]:
!ls "/content/drive/My Drive/nl2ds"

semantic-parser  tweets


Let's load the trigrams first. You can change the below code as you see fit.

In [ ]:
from math import log

bigram_prefix_to_trigram = {}
bigram_prefix_to_trigram_weights = {}

lines = open("/content/drive/My Drive/nl2ds/tweets/covid-tweets-2020-08-10-2020-08-21.trigrams.txt").readlines()
for line in lines:
  word1, word2, word3, count = line.strip().split()
  if (word1, word2) not in bigram_prefix_to_trigram:
    bigram_prefix_to_trigram[(word1, word2)] = []
    bigram_prefix_to_trigram_weights[(word1, word2)] = []
  bigram_prefix_to_trigram[(word1, word2)].append(word3)
  bigram_prefix_to_trigram_weights[(word1, word2)].append(int(count))

# freeup memory
lines = None

## Problem 1.1: Retrieve top next words and their probability given a bigram prefix.

For the following prefixes **word1=middle, word2=of, and n=10**, the output is:



```
a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
...
...
...
```



In [ ]:
import numpy as np
def top_next_word(word1, word2, n=10):
  # write your code here
  all = bigram_prefix_to_trigram[(word1,word2)]
  all_weights = np.array(bigram_prefix_to_trigram_weights[(word1,word2)])
  total_weights = np.sum(np.array(bigram_prefix_to_trigram_weights[(word1,word2)]))
  all_weights_percent = all_weights / total_weights

  topn = all[:n]
  topn_weights_percent = all_weights_percent[:n]
  return topn, topn_weights_percent

next_words, probs = top_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
covid 0.009389671361502348
nowhere 0.008450704225352112
it 0.004694835680751174
lockdown 0.002347417840375587
summer 0.002347417840375587


## Problem 1.2: Sampling n words

Sample next n words given a bigram prefix. Use the probablity distribution defined by the frequency counts. Functions like **numpy.random.choice** will be useful here. Sample without repitition, otherwise all your samples will contain the most frequent trigram.


For the following prefixes **word1=middle, word2=of, and n=10**, the output could be as follows (our outputs may differ):

```
a 0.807981220657277
pandemic 0.023943661971830985
nowhere 0.008450704225352112
the 0.06948356807511737
...
...
...
...
...
```



In [ ]:
def sample_next_word(word1, word2, n=10):
  # write your code here
  all = bigram_prefix_to_trigram[(word1,word2)]
  all_weights = np.array(bigram_prefix_to_trigram_weights[(word1,word2)])
  total_weights = np.sum(np.array(bigram_prefix_to_trigram_weights[(word1,word2)]))
  prob = all_weights / total_weights

  if n <= len(all):
    samples = np.random.choice(all, n, replace=False, p=prob)
  else:
    samples = np.random.choice(all, len(all), replace=False, p=prob)

  indices = []
  for sample in samples:
    indices.append(all.index(sample))
  samples_probs = prob[indices]

  return samples, samples_probs

next_words, probs = sample_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

the 0.06948356807511737
a 0.807981220657277
nowhere 0.008450704225352112
summer 0.002347417840375587
ww2 0.00046948356807511736
global 0.0018779342723004694
pandemic 0.023943661971830985
deadly 0.0009389671361502347
covid 0.009389671361502348
‘ 0.00046948356807511736


## Problem 1.3: Generate sentences starting with a prefix

Generates n-sentences starting with a given sentence prefix. Use [beam search](https://en.wikipedia.org/wiki/Beam_search) to generate multiple sentences. Depending on which method you use to generate next word, you will get different outputs. When you generate <EOS> in a path, stop exploring that path. If you are not careful with your implementation, you may end up in an infinite loop.

If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output is as follows:
```
<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.0001719607222046247
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
...
...
...
```


If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output is as follows:
```
<BOS1> <BOS2> biden calls for a 30 bonus URL #cashgem #cashappfriday #stayathome <EOS> 0.0002495268686322749
<BOS1> <BOS2> biden says all u.s. governors should mandate masks <EOS> 1.6894510541025754e-05
<BOS1> <BOS2> biden says all u.s. governors question cost of a pandemic <EOS> 8.777606198953028e-07
...
...
...
```


If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output may look as follows (since this is sampling, our outputs will difer):

```
<BOS1> <BOS2> trump signs executive orders URL <EOS> 7.150992253427233e-05
<BOS1> <BOS2> trump signs executive actions URL <EOS> 7.117242889600614e-05
<BOS1> <BOS2> trump news president attacked over it <EOS> 1.0546494007903964e-05
<BOS1> <BOS2> trump news president attacked over executive orders URL <EOS> 1.0126405114118984e-05
```

If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output may look as follows:

```
<BOS1> <BOS2> biden harris 2020 <EOS> 0.0015758924114719264
<BOS1> <BOS2> biden harris 2020 URL <EOS> 0.0006443960952032196
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do it URL <EOS> 4.105215709355001e-07
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do our best to stay home <EOS> 1.3158806336098573e-09
...
...
...
...
...
```

Hope you see that sampling gives different outputs compared to deterministically picking the top n-words.


In [ ]:
def generate_sentences(prefix, sampler, beam=10):
  # write your code
  sentences = []
  probs = []
  prefixes = prefix.split()

  ######## at first, we use the given prefix to find #beam of starters. #########
  if sampler == top_next_word:
    sentence_fourth_prefix_list, probas = sampler(prefixes[1], prefixes[2], beam) ##### top_next_words the most probable 10 next words. #####

    each_sentence = []
    for x in sentence_fourth_prefix_list:
      pres_copy = prefixes.copy()
      pres_copy.append(x)
      each_sentence.append(pres_copy)
    ##### each_sentence is repred by [[<BOS1>,<BOS2>,trump,fourth_1],[<BOS1>,<BOS2>,trump,fourth_2]...] at first, then replacing some of them every step

  ######## Since at every step, we always have at most 10 most probable sentences, whenever one of these sentences meets <EOS>,
  ######## we output them and stop this path. Others continuue. ########

    while (len(sentences)<beam): ####### this while loop represents each time step, we need to have a pool of 100 to select from ######
      pool = []
      for sent, prob in zip(each_sentence, probas): ### for every sentence we have 10 next words, put them all into the pool
        next_words, next_probas = sampler(sent[-2], sent[-1], beam)
        candidates_sentences = []
        for x in next_words:
          sent_copy = sent.copy()
          sent_copy.append(x)
          candidates_sentences.append(sent_copy)
        candidates_probs = [prob*p for p in next_probas]
        zipped = list(zip(candidates_sentences,candidates_probs))
        pool.append(zipped)

      flat_pool = [item for sublist in pool for item in sublist]
      sorted_pool = sorted(flat_pool, reverse=True, key = lambda x: x[1])
      top_beam_sentence_prob = sorted_pool[:beam] #### the next 10 most probable sentences selected from sorted pool ####

      each_sentence = [x[0] for x in top_beam_sentence_prob]
      probas = [x[1] for x in top_beam_sentence_prob]

      each_sentence_copy = each_sentence.copy()
      probas_copy = probas.copy()
      for i in range(len(each_sentence_copy)):
        if each_sentence_copy[i][-1] == '<EOS>':
          sentences.append(each_sentence_copy[i])
          probs.append(probas[i])
          each_sentence.remove(each_sentence_copy[i])
          probas.remove(probas_copy[i])

  else:
    sentence_fourth_prefix_list, probas = sampler(prefixes[1], prefixes[2], beam)
    each_sentence = []
    for x in sentence_fourth_prefix_list:
      pres_copy = prefixes.copy()
      pres_copy.append(x)
      each_sentence.append(pres_copy)
    ##### each_sentence is repred by [[<BOS1>,<BOS2>,trump,fourth_1],[<BOS1>,<BOS2>,trump,fourth_2]...] at first, then replacing some of them every step

  ######## Since at every step, we always have at most 10 most probable sentences, whenever one of these sentences meets <EOS>,
  ######## we output them and stop this path. Others continuue. ########

    while (len(sentences)<beam): ####### this while loop represents each time step, we need to have a pool of 100 to select from ######
      pool = []
      for sent, prob in zip(each_sentence, probas): ### for every sentence we have 10 next words, put them all into the pool
        # print(sent)
        next_words, next_probas = sampler(sent[-2], sent[-1], beam)
        candidates_sentences = []
        for x in next_words:
          sent_copy = sent.copy()
          sent_copy.append(x)
          candidates_sentences.append(sent_copy)
        candidates_probs = [prob*p for p in next_probas]
        zipped = list(zip(candidates_sentences,candidates_probs))
        pool.append(zipped)

      flat_pool = [item for sublist in pool for item in sublist]
      sorted_pool = sorted(flat_pool, reverse=True, key = lambda x: x[1])
      top_beam_sentence_prob = sorted_pool[:beam] #### the next 10 most probable sentences selected from sorted pool ####

      each_sentence = [x[0] for x in top_beam_sentence_prob]
      probas = [x[1] for x in top_beam_sentence_prob]

      each_sentence_copy = each_sentence.copy()
      probas_copy = probas.copy()
      for i in range(len(each_sentence_copy)):
        if each_sentence_copy[i][-1] == '<EOS>':
          sentences.append(each_sentence_copy[i])
          probs.append(probas[i])
          each_sentence.remove(each_sentence_copy[i])
          probas.remove(probas_copy[i])


  zipped = zip(sentences, probs)
  zipped = list(zipped)
  res = sorted(zipped, reverse=True, key = lambda x: x[1])

  sentences = [' '.join(x[0]) for x in res]
  probs = [x[1] for x in res]

  return sentences, probs

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)

<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.00010332742295151631
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
<BOS1> <BOS2> trump eyes new unproven coronavirus therapeutic mypillow creator over unproven therapeutic URL <EOS> 8.212549111137046e-05
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over dubious coronavirus drug URL <EOS> 2.3660042627200012e-05
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by ben carson and mypillow founder URL <EOS> 1.2095697936835552e-05
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL via @USER <EOS> 7.432226908194607e-06
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven and dangerous covid-19 treatment URL <EOS> 5.235550241426

# 2. Semantic Parsing

In this part, you are going to build your own virtual assistant! We will be developing two modules: an intent classifier and a slot filler.

In [ ]:
!ls "/content/drive/My Drive/nl2ds/semantic-parser"
parser_files = "/content/drive/My Drive/nl2ds/semantic-parser"

test_answers.txt  test_questions.txt  train_questions_answers.txt


In [ ]:
import json

train_data = []
for line in open(f'{parser_files}/train_questions_answers.txt'):
    train_data.append(json.loads(line))

# print a few examples
for i in range(20):
    print(train_data[i])
    print("-"*80)

{'question': 'Add an album to my Sylvia Plath playlist.', 'intent': 'AddToPlaylist', 'slots': {'music_item': 'album', 'playlist_owner': 'my', 'playlist': 'Sylvia Plath'}}
--------------------------------------------------------------------------------
{'question': 'add Diarios de Bicicleta to my la la playlist', 'intent': 'AddToPlaylist', 'slots': {'playlist': 'Diarios de Bicicleta', 'playlist_owner': 'my', 'entity_name': 'la la'}}
--------------------------------------------------------------------------------
{'question': 'book a table at a restaurant in Lucerne Valley that serves chicken nugget', 'intent': 'BookRestaurant', 'slots': {'restaurant_type': 'restaurant', 'city': 'Lucerne Valley', 'served_dish': 'chicken nugget'}}
--------------------------------------------------------------------------------
{'question': 'add iemand als jij to my playlist named In The Name Of Blues', 'intent': 'AddToPlaylist', 'slots': {'entity_name': 'iemand als jij', 'playlist_owner': 'my', 'playlist'

In [ ]:
test_questions = []
for line in open(f'{parser_files}/test_questions.txt'):
    test_questions.append(json.loads(line))

test_answers = []
for line in open(f'{parser_files}/test_answers.txt'):
    test_answers.append(json.loads(line))

# print a few examples
for i in range(5):
    print(test_questions[i])
    print(test_answers[i])
    print("-"*80)

Add an artist to Jukebox Boogie Rhythm & Blues
{'intent': 'AddToPlaylist', 'slots': {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}}
--------------------------------------------------------------------------------
Will it be rainy at Sunrise in Ramey Saudi Arabia?
{'intent': 'GetWeather', 'slots': {'condition_description': 'rainy', 'timeRange': 'Sunrise', 'city': 'Ramey', 'country': 'Saudi Arabia'}}
--------------------------------------------------------------------------------
Weather in two hours  in Uzbekistan
{'intent': 'GetWeather', 'slots': {'timeRange': 'in two hours', 'country': 'Uzbekistan'}}
--------------------------------------------------------------------------------
Will there be a cloud in VI in 14 minutes ?
{'intent': 'GetWeather', 'slots': {'condition_description': 'cloud', 'state': 'VI', 'timeRange': 'in 14 minutes'}}
--------------------------------------------------------------------------------
add nuba to my Metal Party playlist
{'intent': 

## Problem 2.1: Keyword-based intent classifier

In this part, you will build a keyword-based intent classifier. For each intent, come up with a list of keywords that are important for that intent, and then classify a given question into an intent. If an input question matches multiple intents, pick the best one. If it does not match any keyword, return None.

Caution: You are allowed to look at training questions and answers to come up with a set of keywords, but it is a bad practice to look at test answers.

In [ ]:
# List of all intents
intents = set()
for example in train_data:
    intents.add(example['intent'])
print(intents)

{'GetWeather', 'BookRestaurant', 'AddToPlaylist'}


In [ ]:
keys = set()
for example in train_data:
  if example['intent'] == 'AddToPlaylist':
    keys_of_this_example = example['slots'].keys()
    for key in keys_of_this_example:
      keys.add(key)
print(keys)
print(len(keys))

{'music_item', 'playlist', 'entity_name', 'artist', 'playlist_owner'}
5


In [ ]:
def predict_intent_using_keywords(question):
  # Fill in your code here.
  # intent_keyswords = {}
  BR_keys = ['restaurant_type', 'city', 'served_dish', 'cuisine', 'party_size_number', 'timeRange', 'facility', 'country', 'restaurant_name','sort','party_size_description','state','spatial_relation','poi']
  GW_keys = ['condition_temperature','timeRange','current_location','state','city','country','condition_description','geographic_poi']
  ATP_keys = ['music_item', 'playlist', 'entity_name', 'artist', 'playlist_owner']
  # intent_keyswords['BookRestaurant'] = BR_keys
  # intent_keyswords['GetWeather'] = GW_keys
  # intent_keyswords['AddToPlaylist'] = ATP_keys

  keywords_of_question = test_answers[test_questions.index(question)]['slots'].keys()

  score_BR = 0
  score_GW = 0
  score_ATP = 0
  for key in keywords_of_question:
    if key in BR_keys:
      score_BR += 1
    if key in GW_keys:
      score_GW += 1
    if key in ATP_keys:
      score_ATP += 1
  # if score_BR >= score_GW and score_BR >= score_ATP:
  #   return 'BookRestaurant'
  # elif score_GW >= score_BR and score_GW >= score_ATP:
  #   return 'GetWeather'
  # elif score_ATP >= score_BR and score_ATP >= score_GW:
  #   return 'AddToPlaylist'
  # elif score_ATP==0 and score_BR == 0 and score_GW == 0:
  #   return None

  if score_GW >= score_BR and score_GW >= score_ATP:
    return 'GetWeather'
  elif score_BR >= score_GW and score_BR >= score_ATP:
    return 'BookRestaurant'
  elif score_ATP >= score_BR and score_ATP >= score_GW:
    return 'AddToPlaylist'
  elif score_ATP==0 and score_BR == 0 and score_GW == 0:
    return None

In [ ]:
from collections import Counter

'''Gives intent wise accuracy of your model'''
def evaluate_intent_accuracy(prediction_function_name):
  correct = Counter()
  total = Counter()
  for i in range(len(test_questions)):
    q = test_questions[i]
    gold_intent = test_answers[i]['intent']
    if prediction_function_name(q) == gold_intent:
      correct[gold_intent] += 1
    total[gold_intent] += 1
  for intent in intents:
    print(intent, correct[intent]/total[intent], total[intent])

# Evaluating the intent classifier.
# In our implementation, a simple keyword based classifier has achieved an accuracy of greater than 65 for each intent
evaluate_intent_accuracy(predict_intent_using_keywords)

GetWeather 0.95 100
BookRestaurant 0.97 100
AddToPlaylist 1.0 100


## Problem 2.2: Statistical intent classifier

Now, let's build a statistical intent classifier. Instead of making use of keywords like what you did above, you will first extract features from a given input question. In order to build a feature representation for a given sentence, make use of word2vec embeddings of each word and take an average to represent the sentence. Then train a logistic regression. Feel free to use any libraries you like.

In [ ]:
import nltk
nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [ ]:
from nltk.data import find
import gensim

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
'''Trains a logistic regression model on the entire training data. For an input question (x), the model learns to predict an intent (Y).'''
import numpy as np
from sklearn.linear_model import LogisticRegression

def train_logistic_regression_intent_classifier():
    # Fill in your code here
    input = []
    output = []
    for example in train_data:
      word_list = example['question'].split(' ')
      output.append(example['intent'])
      vectors = []
      for word in word_list:
        try:
          vec = word2vec_model.get_vector(word)
          vectors.append(vec)
        except:
          vectors.append(np.zeros(300))
      vectors = np.array(vectors)
      input.append(np.average(vectors, axis=0))
    input = np.array(input)
    logreg = LogisticRegression()
    logreg.fit(input,output)
    # Feel free to add more cells or functions if needed
    return logreg

In [ ]:
#### train the classifier ####
logreg = train_logistic_regression_intent_classifier()

In [ ]:
'''For an input question, the model predicts an intent'''
def predict_intent_using_logistic_regression(question):
    # Fill in your code here
    word_list = question.split(' ')
    vectors = []
    for word in word_list:
      try:
        vec = word2vec_model.get_vector(word)
        vectors.append(vec)
      except:
        vectors.append(np.zeros(300))
    vectors = np.array(vectors)
    input = np.average(vectors, axis=0)
    intent = logreg.predict(input.reshape(1, -1))
    # Feel free to add more cells or functions if needed
    return intent

In [ ]:
# Evaluate the intent classifier
# Your intent classifier performance will be close to 100 if you have done a good job.
evaluate_intent_accuracy(predict_intent_using_logistic_regression)

GetWeather 0.99 100
BookRestaurant 1.0 100
AddToPlaylist 1.0 100


## Problem 2.3: Slot filling

Build a slot filling model. We will just work with `AddToPlaylist` intent. Ignore other intents.

Hint: No need to rely on machine learning here. You can use ideas like maximum string matching to identify which slots are active and what thier values are. This problem's solution is intentionally left underspecified.

In [ ]:
# Let's stick to one target intent.
target_intent = "AddToPlaylist"

# This intent has the following slots
target_intent_slot_names = set()
for sample in train_data:
    if sample['intent'] == target_intent:
        for slot_name in sample['slots']:
            target_intent_slot_names.add(slot_name)
print(target_intent_slot_names)


# Extract all the relevant questions of this target intent from the test examples.
target_intent_questions = []
for i, question in enumerate(test_questions):
    if test_answers[i]['intent'] == target_intent:
        target_intent_questions.append(question)
print(len(target_intent_questions))

{'music_item', 'playlist', 'entity_name', 'artist', 'playlist_owner'}
100


In [ ]:
#### initiating a database for matching each word of
import re
train_database = {}
for slt in target_intent_slot_names:
  train_database[slt] = set()

for example in train_data:
  if example['intent'] == 'AddToPlaylist':
    slots_of_this_example = example['slots'].keys()
    for slot in slots_of_this_example:
      train_database[slot].add(example['slots'][slot])


In [ ]:
def initialize_slots():
    slots = {}
    for slot_name in target_intent_slot_names:
        slots[slot_name] = None
    return slots

def predict_slot_values(question):
    slots = initialize_slots()
    question = question.lower()
    word_list = question.split(' ')
    for slot_name in target_intent_slot_names:
        # Fill in your code to idenfity the slot value. By default, they are initialized to None.

        if slot_name == 'entity_name':

          # regex = r'(add)|(put)\s(?!the)\s(?P<entityName>[a-z0-9_\s\&\-]+)\s(to)|(into)'
          regex = r'(add|put)\s?(?!the|an|a|this|that)\s?(?P<entityName>[a-z0-9_\s\&\-]+) (to)|(into)'
          pattern = re.compile(regex)
          entname = re.findall(regex,question)
          if len(entname) > 0 and entname[0][1] != '':
            slots[slot_name] = entname[0][1]


        elif slot_name == 'artist':
          # regex = r'(add)|(put)\s(?!the)\s(?P<artist>[a-z0-9_\s\&\-]+)\s(to)|(into)'
          regex = r'(add|put) (?P<artist>[a-z0-9_\s\&\-]+) (to)|(into)'
          pattern = re.compile(regex)
          artst = re.findall(regex,question)
          if len(artst) > 0 and artst[0][1] != '':
            slots[slot_name] = artst[0][1]


        elif slot_name == 'playlist':
          # regex = r'(to)|(into)\s(my|\w\'s)\s(?P<playlist>[a-z0-9_\s]+)\splaylist'
          # regex = r'to (my|the|[a-z\s\-]+\'s)? (?P<playlist>.*) (playlist)?'
          # regex = r'to [a-z]+(\'s)? (?P<playlist>.*) (playlist)?'    this kinda works
          regex = r'to\s?(my|the|[a-z\s\-]+\'s)?\s?(?P<playlist>.*)\s?(playlist)?'
          pattern = re.compile(regex)
          plylst = pattern.findall(question)
          if len(plylst) > 0 and plylst[0][1] != '':
            if 'playlist' in plylst[0][1]:
              slots[slot_name] = plylst[0][1].replace('playlist','').strip('?').strip('.').strip()
            else:
              slots[slot_name] = plylst[0][1]

        else:
          slots[slot_name] = []
          for word in word_list:
            if word in train_database[slot_name]:
              slots[slot_name].append(word)
              word_list.remove(word)
          result = ' '.join(slots[slot_name])
          if result != '':
            slots[slot_name] = result
          else:
            slots[slot_name] = None

    return slots

def evaluate_slot_prediction_recall(slot_prediction_function):
    correct = Counter()
    total = Counter()
    # predict slots for each question
    for i, question in enumerate(target_intent_questions):
        i = test_questions.index(question) # This line is added after the assignment release
        gold_slots = test_answers[i]['slots']
        predicted_slots = slot_prediction_function(question)
        for name in target_intent_slot_names:
            if name in gold_slots:
                total[name] += 1.0
                if predicted_slots.get(name, None) != None and predicted_slots.get(name).lower() == gold_slots.get(name).lower(): # This line is updated after the assignment release
                    correct[name] += 1.0
    for name in target_intent_slot_names:
        print(f"{name}: {correct[name] / total[name]}")


# Our reference implementation got these numbers. You can ask others on Slack what they got.
# music_item 1.0
# playlist 0.67
# artist  0.021739130434782608
# playlist_owner 0.9444444444444444
# entity_name 0.1111111111111111
print("Slot accuracy for your slot prediction model")
evaluate_slot_prediction_recall(predict_slot_values)


Slot accuracy for your slot prediction model
music_item: 0.9636363636363636
playlist: 0.69
entity_name: 0.7222222222222222
artist: 0.34782608695652173
playlist_owner: 0.9259259259259259


In [ ]:
# Find a true positive prediction for each slot
# Fill in your code below along with printing your prediction and gold answer
for i, question in enumerate(target_intent_questions):
  i = test_questions.index(question) # This line is added after the assignment release
  gold_slots = test_answers[i]['slots']
  predicted_slots = predict_slot_values(question)

  for name in target_intent_slot_names:
    if name in gold_slots and predicted_slots[name] != None:
      # print('Groundtruth: ', name, ', Value:', test_answers[i]['slots'][name])
      # print('True Positive Prediction: ', name, ', Value:', predicted_slots[name])
      print('Groundtruth: ', test_answers[i]['slots'], ' Hit:' ,name)
      print('True Positive Prediction: ', name, ', Value:',predicted_slots[name])
      print('#######################################')

Groundtruth:  {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}  Hit: music_item
True Positive Prediction:  music_item , Value: artist
#######################################
Groundtruth:  {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}  Hit: playlist
True Positive Prediction:  playlist , Value: jukebox boogie rhythm & blues
#######################################
Groundtruth:  {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}  Hit: playlist
True Positive Prediction:  playlist , Value: metal party
#######################################
Groundtruth:  {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}  Hit: entity_name
True Positive Prediction:  entity_name , Value: nuba
#######################################
Groundtruth:  {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}  Hit: playlist_owner
True Positive Prediction:  playlist_owner , Value: my
##########################

In [ ]:
# Find a false positive prediction for each slot
# Fill in your code below along with print statement
for i, question in enumerate(target_intent_questions):
  i = test_questions.index(question) # This line is added after the assignment release
  gold_slots = test_answers[i]['slots']
  predicted_slots = predict_slot_values(question)

  for name in target_intent_slot_names:
    if name not in gold_slots and predicted_slots[name] != None:
      print('Groundtruth: ', test_answers[i]['slots'])
      # print('False Positive Prediction: ', name, ', Value:', predicted_slots[name])
      print('False Positive Prediction: ', name, ', Value:',predicted_slots[name])
      print('#######################################')

Groundtruth:  {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
False Positive Prediction:  entity_name , Value: an artist
#######################################
Groundtruth:  {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'}
False Positive Prediction:  artist , Value: an artist
#######################################
Groundtruth:  {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'}
False Positive Prediction:  artist , Value: nuba
#######################################
Groundtruth:  {'music_item': 'album', 'playlist': 'The Sweet Suite'}
False Positive Prediction:  entity_name , Value: the album
#######################################
Groundtruth:  {'music_item': 'album', 'playlist': 'The Sweet Suite'}
False Positive Prediction:  artist , Value: the album
#######################################
Groundtruth:  {'music_item': 'song', 'playlist_owner': 'my', 'playlist': 'madden nfl 16'}
False Positive Prediction:  entity_name 

In [ ]:
# Find a true negative prediction for each slot
# Fill in your code below along with a print statement
for i, question in enumerate(target_intent_questions):
  i = test_questions.index(question) # This line is added after the assignment release
  gold_slots = test_answers[i]['slots']
  predicted_slots = predict_slot_values(question)

  for name in target_intent_slot_names:
    if name not in gold_slots and predicted_slots[name] == None:
      print('Groundtruth: ', test_answers[i]['slots'], 'True Miss: ', name)
      # print('False Positive Prediction: ', name, ', Value:', predicted_slots[name])
      print('True Negative Prediction: ', name, ', Value:',predicted_slots[name])
      print('#######################################')

Groundtruth:  {'music_item': 'artist', 'playlist': 'Jukebox Boogie Rhythm & Blues'} True Miss:  playlist_owner
True Negative Prediction:  playlist_owner , Value: None
#######################################
Groundtruth:  {'entity_name': 'nuba', 'playlist_owner': 'my', 'playlist': 'Metal Party'} True Miss:  music_item
True Negative Prediction:  music_item , Value: None
#######################################
Groundtruth:  {'music_item': 'album', 'playlist': 'The Sweet Suite'} True Miss:  playlist_owner
True Negative Prediction:  playlist_owner , Value: None
#######################################
Groundtruth:  {'music_item': 'song', 'artist': 'Yutaka Ozaki', 'playlist_owner': 'my', 'playlist': 'this is miles davis'} True Miss:  entity_name
True Negative Prediction:  entity_name , Value: None
#######################################
Groundtruth:  {'entity_name': 'give us rest', 'playlist_owner': 'my', 'playlist': '70s Smash Hits'} True Miss:  music_item
True Negative Prediction:  music_it

In [ ]:
# Find a false negative prediction for each slot
# Fill in your code below along with a print statement
# Find a true negative prediction for each slot
# Fill in your code below along with a print statement
for i, question in enumerate(target_intent_questions):
  i = test_questions.index(question) # This line is added after the assignment release
  gold_slots = test_answers[i]['slots']
  predicted_slots = predict_slot_values(question)

  for name in target_intent_slot_names:
    if name in gold_slots and predicted_slots[name] == None:
      print('Groundtruth: ', test_answers[i]['slots'])
      # print('False Positive Prediction: ', name, ', Value:', predicted_slots[name])
      print('False Negative Prediction: ', name, ', Value:',predicted_slots[name] )
      print('#######################################')

Groundtruth:  {'music_item': 'song', 'artist': 'Yutaka Ozaki', 'playlist_owner': 'my', 'playlist': 'this is miles davis'}
False Negative Prediction:  artist , Value: None
#######################################
Groundtruth:  {'music_item': 'song', 'playlist_owner': 'my', 'playlist': 'All Out 60s'}
False Negative Prediction:  playlist , Value: None
#######################################
Groundtruth:  {'playlist_owner': 'my', 'playlist': 'fantastic workout', 'artist': 'sara bareilles'}
False Negative Prediction:  playlist , Value: None
#######################################
Groundtruth:  {'playlist_owner': 'my', 'playlist': 'fantastic workout', 'artist': 'sara bareilles'}
False Negative Prediction:  artist , Value: None
#######################################
Groundtruth:  {'artist': 'Vandemataram Srinivas', 'music_item': 'track', 'playlist': 'HipHop Hot 50'}
False Negative Prediction:  artist , Value: None
#######################################
Groundtruth:  {'playlist': 'travelling'